# Seagull

Authors:  C. L. Adair, D. L. Tucker, with help from L. Jones, J. Carlin, and others

Created:  2024.11.15
Updated: 2025.01.29 (and it has cool stuff in here...I'm impressed with us)

In [ ]:
%pwd

## 1. Initial Setup...

In [ ]:
import lsst.obs.lsst as lsst

In [ ]:
lsst.

In [ ]:
def find_datasets(butler, dataset, collectionPattern="*"):
    for info in butler.collections.query_info(collectionPattern, include_summary=True):
        if dataset in info.dataset_types:
            print(info.name)

find_datasets(butler, "object", collectionPattern="*LSSTCam*")

### 1.1 Import useful python packages

In [ ]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure
from astropy.visualization import make_lupton_rgb


# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import lsst.geom as geom

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

### 1.2 Include user input

In [ ]:
# Which repo, collection, instrument, and skymap to use.
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs
#repo = 'embargo'
repo = '/repo/main'
#collections = 'LSSTComCam/runs/DRP/20241101_20241113/w_2024_46/DM-47566'
#collections = 'LSSTComCam/runs/DRP/20241101_20241120/w_2024_47/DM-47746'
#collections = 'LSSTComCam/runs/DRP/20241101_20241204/w_2024_49/DM-47988'
collections = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235'
collections = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235'
instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'
day_obs_start = 20241101
day_obs_end = 20241230

plotCutouts = True

# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Which CalSpec C26202 spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '~/Downloads/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '~/Downloads/c26202_mod_008.fits'
               }

# RA, DEC of C26202 in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):
raDeg = 106.00
decDeg = -10.50

# List of filters to examine
flist = ['u','g','r','i','z','y']

# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

### 1.3 Define useful classes and functions

In [ ]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
def cutout_im(butler, ra, dec, datasetType, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
    wcs = butler.get('%s.wcs' % datasetType,**dataId)
    xy = geom.PointI(wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get(datasetType, parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def warp_img(ref_img, img_to_warp, ref_wcs, wcs_to_warp):

    config = RegisterConfig()
    task = RegisterTask(name="register", config=config)
    warpedExp = task.warpExposure(img_to_warp, wcs_to_warp, ref_wcs,
                                  ref_img.getBBox())

    return warpedExp

In [ ]:
def make_gif(frame_folder):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"))]
    frame_one = frames[0]
    frame_one.save("animation.gif", format="GIF", append_images=frames,
               save_all=True, duration=500, loop = 0)

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap_name='lsst_cells_v1', cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    skymap = butler.get('skyMap', skymap=skymap_name)


    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId, skymap=skymap_name)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

## 3. Query USDF Butler for ComCam measurements of C26202

### 3.1 Instantiate Butler

In [ ]:
butler = dafButler.Butler(repo, collections=collections)

### 3.2 Find all the `calexp`'s that overlap the sky position of C26202

#### 3.2.1 Find the `dataId`'s for all `calexp`'s in this repo/collection that overlap the RA, DEC of C26202

In [ ]:
datasetRefs = butler.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)

print(f"\nFound {len(datasetRefs)} calexps")

#### 3.2.2 Plot the cutouts for all these `calexp`'s

**Tina, could use add some code here from the DP02_04b_Intermediate_Butler_Queries tutorial notebook so we can view these `calexp` images?  Maybe something from Section 3.1 from that tutorial notebook.  It would be good to take a look at the individual images in case there are any weird `calexp` images that we should ignore.**

In [ ]:
calexp = butler.get('calexp', dataId={'visit': 2024121000374, 'detector': 7})

In [ ]:
calexp_info = calexp.getInfo()

In [ ]:
visit_info = calexp_info.getVisitInfo()
summary_info = calexp_info.getSummaryStats()

In [ ]:
summary_info

In [ ]:
datasetType = 'calexp'
dataId = {'visit': 2024121000374, 'detector': 7}
calexp = butler.get(datasetType, dataId=dataId)

In [ ]:
print(butler.registry.getDatasetType(datasetType))

In [ ]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(calexp.image)
    plt.show()

In [ ]:
bg_calexp = butler.get('calexpBackground', dataId=dataId)

In [ ]:
bg_calexp_image = bg_calexp.getImage()

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(bg_calexp_image)
plt.show()

In [ ]:
skymap = butler.get('skyMap', skymap=skymap_name)


In [ ]:
cutoutSideLength=701
radec = geom.SpherePoint(raDeg, decDeg, geom.degrees)
cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)
xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
patch = tractInfo.getSequentialPatchIndex(patchInfo)
coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': "r"}
parameters = {'bbox': bbox}
#cutout_image = butler.get(datasetType, parameters=parameters, dataId=coaddId)

In [ ]:
cutout_image = butler.get('deepCoadd', parameters=parameters, dataId=coaddId, skymap=skymap_name)

In [ ]:
cutout_image_g = cutout_coadd(butler, raDeg, decDeg, band='g',
                              datasetType='deepCoadd', skymap=skymap, cutoutSideLength=701)
cutout_image_r = cutout_coadd(butler, raDeg, decDeg, band='r',
                              datasetType='deepCoadd', skymap=skymap, cutoutSideLength=701)
cutout_image_z = cutout_coadd(butler, raDeg, decDeg, band='z',
                              datasetType='deepCoadd', skymap=skymap, cutoutSideLength=701)
coadds = [cutout_image_g, cutout_image_r, cutout_image_z]
coadds = MultibandExposure.fromExposures(['g', 'r', 'z'], coadds)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), nrows=1, ncols=2)

rgb_original = create_rgb(coadds.image, bgr=['g', 'r', 'z'], scale=None)
ax[0].imshow(rgb_original, origin='lower')
ax[0].set_title('original', fontsize=30)

ax[1].set_title('re-scaled', fontsize=30)
rgb_scaled = create_rgb(coadds.image, bgr=['g', 'r', 'z'],
                        scale=[0.6, 0.7, 1.0])
ax[1].imshow(rgb_scaled, origin='lower')

ax[0].set_axis_off()
ax[1].set_axis_off()
plt.show()
remove_figure(fig)

In [ ]:
cutout_image_u = cutout_coadd(butler, raDeg, decDeg, band='u',
                              datasetType='deepCoadd', skymap=skymap, cutoutSideLength=701)
cutout_image_g = cutout_coadd(butler, raDeg, decDeg, band='g',
                              datasetType='deepCoadd', skymap=skymap, cutoutSideLength=701)
cutout_image_r = cutout_coadd(butler, raDeg, decDeg, band='r',
                              datasetType='deepCoadd', skymap=skymap, cutoutSideLength=701)
coadds = [cutout_image_u, cutout_image_g, cutout_image_r]
coadds = MultibandExposure.fromExposures(['u', 'g', 'r'], coadds)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), nrows=1, ncols=2)

rgb_original = create_rgb(coadds.image, bgr=['u', 'g', 'r'], scale=None)
ax[0].imshow(rgb_original, origin='lower')
ax[0].set_title('original', fontsize=30)

ax[1].set_title('re-scaled', fontsize=30)
rgb_scaled = create_rgb(coadds.image, bgr=['u', 'g', 'r'],
                        scale=[0.6, 0.7, 1.0])
ax[1].imshow(rgb_scaled, origin='lower')

ax[0].set_axis_off()
ax[1].set_axis_off()
plt.show()
remove_figure(fig)

In [ ]:
cutoutsize = 51 #Defining the size of the cutout box in pixels
visit = 2024121000374
detector = 7

In [ ]:
cutout_calexp = cutout_im(butler, raDeg, decDeg, 'calexp', visit, detector, cutoutSideLength=cutoutsize)

In [ ]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(cutout_calexp.image)
    plt.show()

#### 3.2.3 Create a pandas Dataframe containing the `sourceTable` info for all these `calexp`'s

Now, loop over the `datasetRefs` again, but this time grab the contents of the `sourceTable` table for each `ref` and combine into all into one big pandas DataFrame.  

In [ ]:
src_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('sourceTable', dataId=dataId)
    src_list.append(src)
    print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


Let's look at the result:

In [ ]:
src_all

#### 3.2.4 Save `src_all` as a CSV file

Let's save `src_all` as a CSV file that we can download and examine with TOPCAT:

***(Rename this file to something else???)***

In [ ]:
src_all.to_csv('LSSTComCam_C26202_fields.csv', index=False)

### 3.3 Extract only those rows containing C26202 from the src_all catalog

In [ ]:
# Based on code retrieved from Claude-3.5-Sonnet

# Create a mask to cull sources with "bad" measurements.
mask = (~src_all.pixelFlags_bad) & (~src_all.pixelFlags_saturated) & \
        (~src_all.extendedness_flag) & (src_all.detect_isPrimary)

# Apply mask, keeping only the "good" measurements of `src_all`
src_all_cleaned = src_all[mask]

# Create SkyCoord object for the coordinates of C26202
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

# Create SkyCoord object for all points in the dataframe
df_coords = SkyCoord(ra=src_all_cleaned['ra'].values*u.degree, 
                     dec=src_all_cleaned['dec'].values*u.degree)

# Calculate separations
separations = ref_coord.separation(df_coords)

# Create mask for points within 3.0 arcseconds
mask_sep = separations < 3.0*u.arcsec

# Get filtered dataframe
nearby_good_df = src_all_cleaned[mask_sep]

# If you want to include the separations in the result
orig_columns = nearby_good_df.columns
nearby_good_df = src_all_cleaned[mask_sep].copy()
nearby_good_df['separation_c26202'] = separations[mask_sep].arcsec

# Find (and keep) the closet match within the match radius
best_df = nearby_good_df.sort_values('separation_c26202').drop_duplicates(subset=orig_columns, keep='first')


Add magCalib and magCalibErr columns:

In [ ]:
# Flux in nano-Janskys to AB magnitudes:
best_df['magCalib'] = -2.5*np.log10(best_df['calibFlux']) + 31.4

# Flux error in nano-Janskys to AB magnitude error:
# Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
best_df['magCalibErr'] = 2.5/math.log(10)*best_df['calibFluxErr']/best_df['calibFlux']

Display `visit`, `detector`, `band`, `calibFlux`, `calibFluxErr`, `magCalib`, `magCalibErr`, and `separation_c26202` from best_df, sorted by `visit` and `band`:

In [ ]:
# Set pandas to show all rows...
pd.set_option("display.max_rows", None)

In [ ]:
best_df[['visit', 'detector', 'band', 'calibFlux', 'calibFluxErr', 'magCalib', 'magCalibErr', 'separation_c26202']].sort_values(['visit', 'band'])

In [ ]:
print("""Number of rows:  %d""" % (len(best_df['visit'])))

In [ ]:
# Reset pandas to its default maximum rows to print to screen
pd.reset_option("display.max_rows")

***Do we need to do any further masking/culling in the above table before proceeding?***

***Save to CSV file???***

***(How to name this file???)***

### 3.4 Match pandas dataframe with observed ComCam magnitudes (best_df) with pandas dataframe with the synthetic magnitudes (df_mags)

In [ ]:
# Reset the index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(best_df, df_mags_reset, left_on='band', right_on='index')

combined_df

In [ ]:
print(df_mags)

In [ ]:
# Group by the 'band' column and calculate the median of 'magCalib' for each group
median_values = combined_df.groupby('band')['magCalib'].median().reset_index()
median_values = median_values.rename(columns={'magCalib': 'median_magCalib'})

# Merge the median values back into the combined_df dataframe
combined_df = pd.merge(combined_df, median_values, on='band', how='left')
combined_df

In [ ]:
# Calculate the number of rows for each filter band
row_counts = combined_df.groupby('band').size().reset_index(name='n_total')

# Merge the row counts back into the combined_df dataframe
combined_df = pd.merge(combined_df, row_counts, on='band', how='left')

combined_df

In [ ]:
# Calculate the differences and add the new columns
combined_df['offset_stis'] = combined_df['median_magCalib'] - combined_df['stiswfcnic_007']
combined_df['offset_mod'] = combined_df['median_magCalib'] - combined_df['mod_008']

combined_df

**trying again - this time calculate the median then combine the tables for stis and mod**


In [ ]:
# Calculate the number of rows for each filter band
row_counts = best_df.groupby('band').size().reset_index(name='n_band')

# Merge the row counts back into the combined_df dataframe
combined_df = pd.merge(best_df, row_counts, on='band', how='left')

combined_df

In [ ]:
# Group by the 'n_band' column and calculate the counts of 'band' for each group
count_df = best_df.groupby('band')['magCalib'].count().reset_index()

# Rename the columns for clarity
count_df = count_df.rename(columns={'magCalib': 'n_band'})

count_df

In [ ]:
# Group by the 'band' column and calculate the median of 'magCalib' for each group
median_df = best_df.groupby('band')['magCalib'].median().reset_index()

# Rename the columns for clarity
median_df = median_df.rename(columns={'magCalib': 'median_magCalib'})

median_df

In [ ]:
# Merge the dataframes based on the filter name
combined_df = pd.merge(count_df, median_df, left_on='band', right_on='band')

combined_df

In [ ]:
# Reset the index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(combined_df, df_mags_reset, left_on='band', right_on='index')

combined_df

In [ ]:
# Calculate the differences and add the new columns
combined_df['offset_stis'] = combined_df['median_magCalib'] - combined_df['stiswfcnic_007']
combined_df['offset_mod'] = combined_df['median_magCalib'] - combined_df['mod_008']

combined_df

In [ ]:
print(combined_df)

In [ ]:
# Define the desired order of 'band'
order = ['u', 'g', 'r', 'i', 'z', 'y']

# Remove the 'index' column
combined_df = combined_df.drop(columns=['index'])

# Reorder the dataframe based on the 'band' column
combined_df['band'] = pd.Categorical(combined_df['band'], categories=order, ordered=True)
combined_df = combined_df.sort_values('band').reset_index(drop=True)

combined_df

***Save results to CSV file???***

***(How to name this file???)***

**Let's stop here for now:**

In [ ]:
raise StopExecution

## 4. Measure differences between the Observed ComCam and the LSST Synthetic Mags for C26202

***Calculate statistics from the matched dataframe from Section 3 above.***

In [ ]:
# DES DR2 AB offsets based on c26202_stisnic_007.fits from William Wester's DES-doc#15451...

print "AB offsets based on c26202_stisnic_007.fits"
print "==========================================="
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_wavg-r_wavg) - (g_ww-r_ww)
aboffset_ri = (r_wavg-i_wavg) - (r_ww-i_ww)
aboffset_iz = (i_wavg-z_wavg) - (i_ww-z_ww)
aboffset_zY = (z_wavg-Y_wavg) - (z_ww-Y_ww)

print "WAVG offsets"
print "------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_auto-r_auto) - (g_ww-r_ww)
aboffset_ri = (r_auto-i_auto) - (r_ww-i_ww)
aboffset_iz = (i_auto-z_auto) - (i_ww-z_ww)
aboffset_zY = (z_auto-Y_auto) - (z_ww-Y_ww)

print "MAG_AUTO offsets"
print "----------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 


## 5. Sandbox

In [ ]:
datasetRefs = butler.query_datasets("visitSummary", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

#print(datasetRefs)

print(f"\nFound {len(datasetRefs)} calexps")

In [ ]:
# Retrieve sourceTable for this visit & detector...

datasetType = 'sourceTable'
#dataId = {'visit': visit, 'detector': detector}
dataId = ref.dataId['visit']
dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
print(dataId)

src = butler.get(datasetType, dataId=dataId)

print(f"Retrieved catalog of {len(src)} sources.")

In [ ]:
butler.registry.queryDataIds(dimensions=('exposure'))

In [ ]:
icExp = butler.get('icExp', dataId=dataId)

In [ ]:
icExp_info = icExp.getInfo()

In [ ]:
print(icExp_info.getMetadata())

In [ ]:
print(dataId)

In [ ]:
datasetRefs = butler.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*src*')):
    print(dt)

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*icSrc*')):
    print(dt)

In [ ]:
icSrc = butler.get('icSrc', dataId=dataId)

In [ ]:
icSrc.asAstropy()

In [ ]:
src1 = butler.get('src', dataId=dataId)

In [ ]:
src1.asAstropy()